##1) 安裝套件（指定版本範圍＋用途註解）

In [1]:
# 安裝 LINE Bot SDK v3（官方最新主線）
!pip install "line-bot-sdk>=3.0.0,<4.0.0"

# 安裝 Flask（Web 框架，提供 webhook 端點）
!pip install "Flask>=3.0.0,<4.0.0"

# 安裝 pyngrok（把本機/Colab 5000 port 映射成公開 HTTPS）
!pip install "pyngrok>=7.0.0,<8.0.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.7/816.7 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 12.0 MB/s eta 0:00:00


##2) 開 ngrok（先執行這一格拿到公開網址）

In [2]:
from pyngrok import ngrok

# 換成你自己的 ngrok authtoken
NGROK_AUTHTOKEN = "2nh1S1yuCNhyHz7E9kibC5G9jfJ_5poDL8LVgJgfAm2a1ZJRc"   # 例：2pEWt4...（不要公開在 GitHub）
PORT = 5000

# 設定金鑰
ngrok.set_auth_token(NGROK_AUTHTOKEN)

# 關閉舊隧道（避免重複）
for t in ngrok.get_tunnels():
    ngrok.disconnect(t.public_url)

# 建立新隧道，取得公開 HTTPS 網址
public_url = ngrok.connect(PORT, "http").public_url
print("👉 外網網址：", public_url)
print("Webhook URL：", public_url + "/callback")


👉 外網網址： https://35e5049ea40e.ngrok-free.app
Webhook URL： https://35e5049ea40e.ngrok-free.app/callback


##3) LINE Bot Echo 程式（v3 SDK 正式寫法）

In [3]:
#從Line bot sdk pypl網站找到程式碼貼上
#https://pypi.org/project/line-bot-sdk/
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token='+Ri09o6iK7SXuNUaZEjWtlf9mZLp1xAt8NPXDVCN0shGvq1wvris4Oo29FZWdNqm7BW5a559L0klMfDltAo4qvsr1GI7OXMW2lW/mcZZYd2pwDpjrPQdxmKQ5W3vpYRC1VSq4rM84ksy+BybGREwygdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('4bc05bd9e903340338507195040bd2ff')


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text)]
            )
        )

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [29/Sep/2025 06:45:18] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Sep/2025 06:47:40] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Sep/2025 06:48:03] "POST /callback HTTP/1.1" 200 -
